[LangChain Docummentation](https://python.langchain.com/en/latest/index.html)

需先安裝openAI、LangChain

In [ ]:
!pip install langchain
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.2/858.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.0 MB/s eta 0:00:00


### 載入OpenAI key

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 'sk-'

[modules](https://python.langchain.com/en/latest/modules/models.html): 為 Langchain 當中支持的各種大型語言模型，如：OpenAI、HuggingFace、cohere 等。


[prompt](https://python.langchain.com/en/latest/modules/prompts.html): 進行 Prompt 管理，序列化。

[chain](https://python.langchain.com/en/latest/modules/chains.html): LangChain 為 Chain 提供標準接口、與其他工具的大量集成以及用於常見應用程序的端到端鏈。

[Agent](https://python.langchain.com/en/latest/modules/agents.html): LangChain 為 Agent 提供了一個標準接口，可供選擇的代理選擇，以及端到端代理的示例。例如："zero-shot-description" 等

[Memory](https://python.langchain.com/en/latest/modules/memory.html): LangChain 提供了 Memory 標準接口、 Memory 實現的集合以及使用內存的鏈/代理的示例。

## Models: 透過語言模型，取得摘要、推理、生成能力

LangChain 中最基本的元素 Model，底下可以透過簡單的例子示範如何操作。 為此，讓我們假設一下正在打造一個服務，專門提供公司名稱，只要給我們公司描述，就給出建議的公司命名。

為了可以實現這個目標，我們首先 import OpenAI 來執行，model-name 可以選擇 'text-davinci-003'，結束後，基本上你會覺得跟操作 ChatGPT 類似，如此一來你就完成了一個 LLM 應用程式。

In [ ]:
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model_name="text-davinci-003")

In [ ]:
text = "Give a good company name for that makes colorful shoes?"
print(llm(text))



RainbowSolez


## Prompt: 替大型語言模型(LLM) 管理提示語(prompt)

常在應用程式中呼叫 LLM 時，你不會直接將使用者撰寫的文字傳送至 LLM。相反的，你可能將使用者撰寫的文字進行組合，再傳送至 LLM 當中。

例如，在前面的例子中，我們是要給定公司產品後，產生出合適的公司名稱。既然如此，我們可以把公司產品名稱作為參數，以此打造一個 Prompt。

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"], #input 變數
    template="What is a good name for a company that makes {product}?",  #prompt
)

In [ ]:
print(prompt.format(product="colorful shoes"))

What is a good name for a company that makes colorful shoes?


In [ ]:
print(llm(prompt.format(product="colorful shoes")))



Rainbow Footwear.


## Chains: 結合 LLM 與 prompts 於多個步驟工作流中

在 LangChain 中，Chain 可以是由 llm 或是其他 Chain 連結接在一起。

最核心的 Chain 是 LLMChain，它由 PromptTemplate 和 LLM 結合。

延續前面了範例，這個環節中，我們建構一個 LLMChain，這個 Chain 會接受使用者輸入，並且使用 PromptTemplate 將其規格化，最後將規格化內容傳遞給 LLM。

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)



In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
chain.run("colorful shoes")

'\n\nRainbow Footwear.'

In [ ]:
from typing import Dict, List
from langchain.chains.base import Chain
class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain
    @property
    def input_keys(self) -> List[str]:
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [ ]:
prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="What is a good slogan for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("colorful socks")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Kaleidoscope Kicks.

"Brighten up your day with Colorful Socks!"


## Agents: 讓 LLM 選擇要用的 tool 解決問題

到目前為止，看到的應用都是是按照預定的順序運行的，特別是像 Chain 用法。

然而，Agnet 不再如此，能夠使用 LLM 來確定要採取的動作及順序。動作可以是使用工具並觀察其輸出，也可以是返回結果給用戶。如果正確使用， Agent 可以非常強大。

接觸 Agnets ，應該會了解以下概念：

* tool：執行特定職責的功能。這可以是 Google搜索、查找資料庫、Python REPL、其他Chain 等等。
* LLM：大型語言模型提供 Agents 決定要執行的動作以及順序。
* Agent：要使用的代理。

In [ ]:
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=8b7595ec69b99c93c5ab7113a0c5a6dbfba1310aea0d6b8de1f4055177cdb29f
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = "169f6c497ae97c71bfe15b079d28cf315ca9eabbfad75c60b520c25474a323a9"

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
# 第一步，讀取 llm 用來提供 Agent 做事情
llm = OpenAI(temperature=0)
# 第二步，讓我們添加些 tool 來提供動作，這邊選擇網路搜尋功能的 serpapi 以及計算數字用的 llm-math
tools = load_tools(["serpapi", "llm-math"], llm=llm)
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# Now let's test it out!
agent.run("What was the high temperature in Taiwan yesterday in Kaohsiung? What is that number different with the high temperature in Taiwan yesterday in Taipei?")



> Entering new AgentExecutor chain...
 I need to find out the temperatures in both Kaohsiung and Taipei.
Action: Search
Action Input: "High temperature in Taiwan yesterday Kaohsiung Taipei"
Observation: Kaohsiung Weather History for the Previous 24 Hours ; 82 °F · 81 °F · 81 °F ...
Thought: I need to compare the two temperatures
Action: Calculator
Action Input: 82 - 81
Observation: Answer: 1
Thought: I now know the final answer
Final Answer: The high temperature in Kaohsiung yesterday was 1 degree higher than the high temperature in Taipei yesterday.

> Finished chain.


'The high temperature in Kaohsiung yesterday was 1 degree higher than the high temperature in Taipei yesterday.'

## Memory: 賦予 Chains 與 Agents 記憶

Chain 以及 Agent 都是 stateless 的。但通常，我們可能會希望添加 "記憶" 的概念在其中，以便他可以記住先前的互動資訊。最簡單的例子就是在設計聊天機器人時，你希望他會記住過往的對話，善用上下文來進行更流暢的對談。簡單來說，這將是一種短期記憶。在複雜情況下，你可能會需要 Chain 或是 Agent 隨著時間的推移記住重要的資訊，這將是一種長期記憶。

在 LangChain 當中，為了賦予記憶的概念，專門建造了幾種 Chain。以下將示範 ConversationChain，預設情況下 ConversationChain 具有一種簡單的記憶類型，能夠記住「所有」先前聊天過程，並添加在傳遞的上下文中。

In [ ]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hi there! It's nice to meet you. How can I help you today?


In [ ]:
output = conversation.predict(input="I am doing well! Currently, just have a conversation with you, an AI.")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I am doing well! Currently, just have a conversation with you, an AI.
AI:

> Finished chain.
 That's great! It's nice to have a conversation with you too. What would you like to talk about?
